In [ ]:
from nbdev import *
%nbdev_default_export examples

Cells will be exported to pct.examples,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

# Examples

> Examples of the PCT library in use.

In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
import gym 
import os

In [ ]:
os = os.name

## Cartpole

0       Cart Position             
1       Cart Velocity             
2       Pole Angle                
3       Pole Angular Velocity 
        


In [ ]:
from pct.hierarchy import PCTHierarchy
from pct.functions import OpenAIGym
from pct.functions import IndexedParameter
from pct.functions import Integration
from pct.functions import GreaterThan

Create a hierarchy of 4 levels each with one node.

In [ ]:
cartpole_hierarchy = PCTHierarchy(levels=4, cols=1, name="cartpoleh")

In [ ]:
cartpole = OpenAIGym("CartPole-v1", name="CartPole-v1", render=False)

Create functions for each of the observation parameters of the Cartpole environment. Insert them into the hierarchy at the desired places.

In [ ]:
cartpole_hierarchy.insert_function(level=0, col=0, collection="perception", function=IndexedParameter(index=1, name="cart_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=1, col=0, collection="perception", function=IndexedParameter(index=0, name="cart_position", links=[cartpole]))
cartpole_hierarchy.insert_function(level=2, col=0, collection="perception", function=IndexedParameter(index=3, name="pole_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=3, col=0, collection="perception", function=IndexedParameter(index=2, name="pole_velocity", links=[cartpole]))

In [ ]:
cartpole_hierarchy.insert_function(level=0, col=0, collection="output", function=Integration(gain=2, slow=10, name="force", links='subtract'))

Link the output of the hierarchy back to the Cartpole environment.

Add the cartpole function as one that is executed before the actual hierarchy.

In [ ]:
cartpole_hierarchy.add_preprocessor(cartpole)

Add a post function to convert the output to 1 or 0 as required by the Cartpole environment. 

In [ ]:
cartpole_hierarchy.add_postprocessor(GreaterThan(threshold=0, upper=0, lower=1, links=[]))

In [ ]:
link = cartpole_hierarchy.get_output_function()
cartpole.add_link(link)

In [ ]:
cartpole_hierarchy.summary(build=False)

cartpoleh PCTHierarchy
**************************
PRE: CartPole-v1 OpenAIGym | 0 | links  greaterthan 
Level 0
level0col0 PCTNode
----------------------------
REF: constant Constant | 1 | links  proportional1 
PER: cart_velocity IndexedParameter | index 1 | 0 | links  CartPole-v1 
COM: subtract Subtract | 0 | links  constant variable 
OUT: force Integration | gain 2 slow 10  | 0 | links  subtract 
----------------------------
Level 1
level1col0 PCTNode
----------------------------
REF: constant1 Constant | 1 | links  proportional2 
PER: cart_position IndexedParameter | index 0 | 0 | links  CartPole-v1 
COM: subtract1 Subtract | 0 | links  constant1 variable1 
OUT: proportional1 Proportional | gain 10 | 0 | links  subtract1 
----------------------------
Level 2
level2col0 PCTNode
----------------------------
REF: constant2 Constant | 1 | links  proportional3 
PER: pole_velocity IndexedParameter | index 3 | 0 | links  CartPole-v1 
COM: subtract2 Subtract | 0 | links  constant2 variable2 

Sit back and observe the brilliance of your efforts. 

In [ ]:
cartpole_hierarchy.summary()

cartpoleh PCTHierarchy
**************************
PRE: CartPole-v1 OpenAIGym | 0 | links  greaterthan 
Level 0
level0col0 PCTNode
----------------------------
REF: constant Constant | 1 | links  proportional1 
PER: cart_velocity IndexedParameter | index 1 | 0 | links  CartPole-v1 
COM: subtract Subtract | 0 | links  constant variable 
OUT: force Integration | gain 2 slow 10  | 0 | links  subtract 
----------------------------
Level 1
level1col0 PCTNode
----------------------------
REF: constant1 Constant | 1 | links  proportional2 
PER: cart_position IndexedParameter | index 0 | 0 | links  CartPole-v1 
COM: subtract1 Subtract | 0 | links  constant1 variable1 
OUT: proportional1 Proportional | gain 10 | 0 | links  subtract1 
----------------------------
Level 2
level2col0 PCTNode
----------------------------
REF: constant2 Constant | 1 | links  proportional3 
PER: pole_velocity IndexedParameter | index 3 | 0 | links  CartPole-v1 
COM: subtract2 Subtract | 0 | links  constant2 variable2 

In [ ]:
cartpole_hierarchy(verbose=True)

-0.021 -0.230 0.036 0.286 1.0 False {} 
level0col0 1.000 -0.230 1.000 0.200 
level1col0 1.000 -0.021 1.000 10.000 
level2col0 1.000 0.286 1.000 10.000 
level3col0 1.000 0.036 1.000 10.000 


IndexError: list index out of range

In [ ]:
cartpole.close()